<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [8]:
# run this on top of all of Your notebooks -- it does all the dirty imports.
exec(open('initialize_jupyter_handybeam.py').read())
import strathclyde

In [71]:
# run the process once to see the data output format
focal_point=(0.0,30e-3,40e-3)
focal_radius=np.sqrt(focal_point[0]**2+focal_point[1]**2+focal_point[2]**2)
importlib.reload(strathclyde) # this makes sure that any changes to the source code are being used in this session
array_builder=strathclyde.LinearArray(
    radiation_frequency=500e3,
    sound_velocity=5600,
    sampling_density=11,
    passive_aperture=32e-3,
    element_pitch=3e-3,
    element_width=1e-3,
    element_count=16,
    focal_point=focal_point,
    amplitude_setting=1e4)
array_builder

Basic linear probe:
> Environment:
>>   radiation frequency: 500.0kHz
>>   sound_velocity :5600.0m/s
>>   sound wave length :11.200mm
>>   medium wavenumber: 89.3[waves/meter]
>>   point sources sampling density: 11pt/lambda linear, spacing of 1.018mm
  
> Probe definition:
>>   Passive aperture: 32.0mm
>>   element width: 1.000mm
>>   element count: 16
  
> Probe calculated properties:
>>   inter-element gap: 2.0mm
>>   Active aperture: 48.0mm
>>   Active aperture near field transition/boundary: 51.4mm
>>   Passive aperture near field transition/boundary: 22.9mm
  
> Focal point calculated properties:
>>   focal distance: 50.0mm
>>   active aperture -6dB focal spot size: 11.9mm
>>   passive aperture -6dB natural focus spot size: 8.2mm

In [72]:
array_builder.stats

{'radiation_frequency': 500000.0,
 'sound_velocity': 5600,
 'wavelength': 0.0112,
 'wavenumber': 89.28571428571429,
 'sampling_density_setting': 11,
 'sampling_density_effective': 0.0010181818181818183,
 'passive_aperture': 0.032,
 'element_width': 0.001,
 'element_count': 16,
 'element_gap': 0.002,
 'active_aperture': 0.048,
 'active_aperture_nearfield_boundary': 0.05142857142857143,
 'passive_aperture_nearfield_boundary': 0.022857142857142857,
 'current_focal_point_distance': 0.05,
 'active_aperture_focus_power_estimate': 0.011899999999999999,
 'passive_aperture_focus_power_estimate': 0.008159999999999999}

In [14]:
# load the library with general purpose minimizers
import scipy.optimize
# see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize

In [76]:
# quest: optimize passive_aperture to achieve passive_aperture_nearfield_boundary of 1.0

passive_aperture_nearfield_boundary_target=2.0

# define cost function:

def costFunction(adjustable_variable):
    # do the calculation
    array_builder=strathclyde.LinearArray(
                radiation_frequency=500e3,
                sound_velocity=5600,
                sampling_density=11,                
                passive_aperture=adjustable_variable, # ! note: adjustable_variable=passive_aperture
                element_pitch=3e-3,
                element_width=2e-3,
                element_count=16,
                focal_point=focal_point,
                amplitude_setting=1e4)
    stats=array_builder.stats    
    
    # Core: The cost function
    cost=np.abs(stats.passive_aperture_nearfield_boundary - passive_aperture_nearfield_boundary_target)
    
    return cost

# define a starting point
passive_aperture_nearfield_boundary_starting_point=50e-3
starting_point=passive_aperture_nearfield_boundary_starting_point # starting point 
tolerance_on_adjustable_variable=1e-4
tolerance_on_cost_function=1e-4

result = scipy.optimize.minimize(costFunction, starting_point, method='Nelder-Mead',
             options={'xatol': tolerance_on_adjustable_variable, 'fatol': tolerance_on_cost_function, 'disp': True})

result

Optimization terminated successfully.
         Current function value: 0.000477
         Iterations: 17
         Function evaluations: 34


 final_simplex: (array([[0.29929688],
       [0.299375  ]]), array([0.00047725, 0.00056676]))
           fun: 0.0004772458757560383
       message: 'Optimization terminated successfully.'
          nfev: 34
           nit: 17
        status: 0
       success: True
             x: array([0.29929688])

In [90]:
# verify that the result is correct -- supply the result.x[0] to the array_builder
array_builder=strathclyde.LinearArray(
    radiation_frequency=500e3,
    sound_velocity=5600,
    sampling_density=11,
    #passive_aperture=32e-3,
    passive_aperture=result.x[0], # ! Use the result from the cost function minimiser
    element_pitch=3e-3,
    element_width=2e-3,
    element_count=16,
    focal_point=focal_point,
    amplitude_setting=1e4)
print('for passive_aperture={:0.2f}mm: effective passive_aperture_nearfield_boundary={:0.2f}mm'.format(1e3*result.x[0], 1e3*array_builder.stats.passive_aperture_nearfield_boundary))
print('')
array_builder

for passive_aperture=299.30mm: effective passive_aperture_nearfield_boundary=1999.52mm



Basic linear probe:
> Environment:
>>   radiation frequency: 500.0kHz
>>   sound_velocity :5600.0m/s
>>   sound wave length :11.200mm
>>   medium wavenumber: 89.3[waves/meter]
>>   point sources sampling density: 11pt/lambda linear, spacing of 1.018mm
  
> Probe definition:
>>   Passive aperture: 299.3mm
>>   element width: 2.000mm
>>   element count: 16
  
> Probe calculated properties:
>>   inter-element gap: 1.0mm
>>   Active aperture: 48.0mm
>>   Active aperture near field transition/boundary: 51.4mm
>>   Passive aperture near field transition/boundary: 1999.5mm
  
> Focal point calculated properties:
>>   focal distance: 50.0mm
>>   active aperture -6dB focal spot size: 11.9mm
>>   passive aperture -6dB natural focus spot size: 76.3mm